# Module 1 — Quantum Computing Understanding (Advanced Labs)

**Purpose:** Build deep intuition about qubits, superposition, interference, entanglement, teleportation, and state visualization. This notebook contains step-by-step explanations, runnable code (Qiskit), and exercises for each lab.

**Pre-requisites:** Python 3.9+, install dependencies:
```
pip install qiskit qiskit-aer matplotlib numpy
```


In [1]:
pip install qiskit qiskit-aer matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 135.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 138.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 145.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 113.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 175.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 150.4 MB/s eta 0:00:0000:01
   ━━

## Lab 1.1 — Multi-qubit Superposition & Interference

**Objective:** Create multi-qubit superposition states, apply phases, and observe interference patterns. We will use a 3-qubit example and step through how phase rotations change measurement probabilities.

In [3]:
pip install qiskit-aer

Note: you may need to restart the kernel to use updated packages.


## Superposition on 3 Qubits
**Explanation:** This block demonstrates superposition. Applying the Hadamard gate to each qubit changes them from definite states to combinations of |0> and |1>. The 3-qubit system enters a superposition of all 8 possible states. Measurement collapses this superposition into classical outcomes, approximately uniformly distributed. This illustrates quantum parallelism.

In [12]:
# Lab 1.1 – Equal superposition on 3 qubits
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

#Create a 3-qubit circuit with 3 classical bits
qc = QuantumCircuit(3, 3)

#Apply Hadamard gates to all qubits → equal superposition
qc.h([0, 1, 2])
qc.barrier()
qc.measure([0, 1, 2], [0, 1, 2])

#Initialize the Aer simulator
sim = AerSimulator()

#Transpile for simulator
compiled = transpile(qc, sim)

#Run the circuit
job = sim.run(compiled, shots=2048)
result = job.result()

#Get counts and visualize
counts = result.get_counts()
print("Counts for equal superposition (should be ~uniform):")
print(counts)

plot_histogram(counts)
plt.show()

Counts for equal superposition (should be ~uniform):
{'110': 250, '001': 274, '100': 233, '010': 253, '000': 274, '011': 229, '111': 267, '101': 268}


### Add phase and re-interfere
We will mark a target state by applying a phase (P gate) on some qubits and then apply Hadamards again to see constructive/destructive interference. This is similar to how oracles + diffusion work in algorithms like Grover.

## Phase and Interference

**Explanation:** A phase shift is applied to one qubit, followed by Hadamard gates. This modifies probability amplitudes, causing constructive and destructive interference. Some outcomes are amplified and others suppressed. Students learn that quantum gates influence amplitudes, not just probabilities.

In [13]:
# Create the circuit
qc2 = QuantumCircuit(3, 3)

# Prepare equal superposition
qc2.h([0, 1, 2])

# Apply a phase of π to qubit 2 (like marking |101>)
qc2.p(np.pi, 2)

# Add barrier and apply H again for interference
qc2.barrier()
qc2.h([0, 1, 2])

# Measure all qubits
qc2.measure([0, 1, 2], [0, 1, 2])

# Create the Aer simulator
sim = AerSimulator()

# Transpile for simulator
compiled_qc2 = transpile(qc2, sim)

# Run and collect results
job2 = sim.run(compiled_qc2, shots=2048)
counts2 = job2.result().get_counts()

# Display results
print("Counts after phase and H (interference):")
print(counts2)
plot_histogram(counts2)
plt.show()

Counts after phase and H (interference):
{'100': 2048}


**Notes & exercise:**
- Try different phase angles (pi/2, pi/4).
- Implement a multi-qubit controlled phase (use mct or controlled-Z constructions) to mark a single basis state exactly.
- Observe how probabilities concentrate or cancel.

## Lab 1.2 — GHZ vs W states (3-qubit entanglement)

**Objective:** Prepare GHZ and W states, compare their measurement statistics and robustness to qubit loss.

**Theory summary:**
- GHZ: (|000> + |111>)/√2 — strong global correlations; if one qubit is lost, entanglement collapses.
- W: (|001> + |010> + |100>)/√3 — remains partially entangled if one qubit is lost.


## GHZ State Preparation

**Explanation:** The GHZ state is a 3-qubit maximally entangled state, producing either all 0s or all 1s on measurement. A Hadamard gate is applied to one qubit, and CNOTs entangle the other two. Measurement of one qubit determines the others, demonstrating entanglement.

In [14]:
# GHZ state preparation and measurement – Updated for Qiskit ≥ 1.0

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Create a 3-qubit GHZ circuit
qc_ghz = QuantumCircuit(3, 3)
qc_ghz.h(0)          # Put qubit 0 into superposition
qc_ghz.cx(0, 1)      # Entangle qubit 1 with qubit 0
qc_ghz.cx(0, 2)      # Entangle qubit 2 with qubit 0
qc_ghz.barrier()
qc_ghz.measure([0, 1, 2], [0, 1, 2])

# Initialize Aer simulator
sim = AerSimulator()

# Transpile for simulator
compiled_ghz = transpile(qc_ghz, sim)

# Run simulation
job_ghz = sim.run(compiled_ghz, shots=4096)
result = job_ghz.result()
counts_ghz = result.get_counts()

# Display results
print("GHZ counts: (expect mostly 000 and 111)")
print(counts_ghz)
plot_histogram(counts_ghz)
plt.show()

GHZ counts: (expect mostly 000 and 111)
{'111': 2049, '000': 2047}


## W-State Approximate Construction

**Explanation:** The W-state distributes a single excitation across qubits: |001>, |010>, or |100>. Unlike GHZ, it spreads amplitude evenly, showing different entanglement types. Students learn how entanglement can vary in structure and robustness.

In [15]:
# W-state approximate construction and measurement (Qiskit ≥ 1.0)

import math
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Initialize simulator
sim = AerSimulator()

# Create 3-qubit circuit
qc_w = QuantumCircuit(3, 3)

# Known decomposition: rotate then cascade CNOTs to distribute amplitude
theta1 = 2 * math.acos(1 / math.sqrt(3))
qc_w.ry(theta1, 0)
qc_w.cx(0, 1)

theta2 = 2 * math.acos(1 / math.sqrt(2))
qc_w.ry(theta2, 1)
qc_w.cx(1, 2)

qc_w.barrier()
qc_w.measure([0, 1, 2], [0, 1, 2])

# Transpile and run
compiled_w = transpile(qc_w, sim)
job_w = sim.run(compiled_w, shots=4096)

# Get results
result = job_w.result()
counts_w = result.get_counts()

# Display output
print("W counts (approx, expect 001, 010, 100):")
print(counts_w)
plot_histogram(counts_w)
plt.show()

W counts (approx, expect 001, 010, 100):
{'001': 1431, '000': 656, '110': 652, '111': 1357}


**Exercise:** Remove one qubit measurement (simulate tracing out) and see how the marginal distribution for the remaining two qubits differs between GHZ and W.

## Lab 1.3 — Quantum Teleportation (complete, with classical correction)

**Objective:** Teleport an arbitrary single-qubit state from qubit 0 to qubit 2 using an entangled pair and two classical bits. We'll demonstrate the full flow and verify by statevector comparison (deterministic on simulator).

## Quantum Teleportation

**Explanation:** This demonstrates teleporting a qubit state using entanglement and classical communication. The sender qubit is entangled with a pair shared with the receiver. Bell measurements are performed, and measurement results sent classically allow corrective operations on the receiver. Verification confirms the original state is reconstructed. It teaches the combination of quantum and classical processes.

In [29]:
# Quantum Teleportation (Qiskit ≥ 1.0) — Corrected

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector, partial_trace, state_fidelity, DensityMatrix
import matplotlib.pyplot as plt

# --- Prepare an arbitrary single-qubit state (H then T) ---
prep = QuantumCircuit(1)
prep.h(0)
prep.t(0)
state = Statevector.from_instruction(prep)
print("Original single-qubit statevector:")
print(state.data)

# --- Build teleportation circuit (3 qubits, 2 classical bits) ---
qc_tel = QuantumCircuit(3, 2)
qc_tel.h(0)
qc_tel.t(0)
qc_tel.h(1)
qc_tel.cx(1, 2)
qc_tel.cx(0, 1)
qc_tel.h(0)
qc_tel.measure(0, 0)
qc_tel.measure(1, 1)

print("\nTeleportation circuit:")
print(qc_tel.draw())

# --- Simulate circuit with qasm simulator ---
sim = AerSimulator()
compiled_circuit = transpile(qc_tel, sim)
job = sim.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()
print("\nMeasurement results (sender bits):")
print(counts)

plot_histogram(counts)
plt.show()

# --- Statevector verification (no measurement, apply corrections) ---
qc_verify = QuantumCircuit(3)
qc_verify.h(0)
qc_verify.t(0)
qc_verify.h(1)
qc_verify.cx(1, 2)
qc_verify.cx(0, 1)
qc_verify.h(0)
qc_verify.cx(1, 2)  # correction X
qc_verify.cz(0, 2)  # correction Z

# Full 3-qubit statevector
sv_final = Statevector.from_instruction(qc_verify)

# Reduce to receiver qubit (qubit 2) → returns DensityMatrix
receiver_rho = partial_trace(sv_final, [0, 1])
print("\nReceiver qubit density matrix:")
print(receiver_rho.data)

# Compute fidelity with original state
fidelity = state_fidelity(DensityMatrix(state), receiver_rho)
print(f"\nFidelity between original and teleported state: {fidelity:.6f}")

# Optional: visualize receiver qubit on Bloch sphere
# Can pass DensityMatrix directly
plot_bloch_multivector(receiver_rho)
plt.show()

Original single-qubit statevector:
[0.70710678+0.j  0.5       +0.5j]

Teleportation circuit:
     ┌───┐┌───┐     ┌───┐┌─┐
q_0: ┤ H ├┤ T ├──■──┤ H ├┤M├
     ├───┤└───┘┌─┴─┐└┬─┬┘└╥┘
q_1: ┤ H ├──■──┤ X ├─┤M├──╫─
     └───┘┌─┴─┐└───┘ └╥┘  ║ 
q_2: ─────┤ X ├───────╫───╫─
          └───┘       ║   ║ 
c: 2/═════════════════╩═══╩═
                      1   0 

Measurement results (sender bits):
{'01': 260, '00': 261, '10': 254, '11': 249}

Receiver qubit density matrix:
[[0.5       +0.00000000e+00j 0.35355339-3.53553391e-01j]
 [0.35355339+3.53553391e-01j 0.5       -6.93334780e-33j]]

Fidelity between original and teleported state: 1.000000


**Instructor note:** For clarity in class, demonstrate one branch explicitly: if measurement results are `00` then no correction; if `10` apply Z; if `01` apply X; if `11` apply X and Z (or Z and X). Show how qubit 2 recovers the original state.

## Lab 1.4 — Bloch-sphere visualization & measurement in different bases

**Objective:** Visualize single-qubit states on the Bloch sphere and perform measurements in X and Y bases.


## Bloch Sphere Visualization and X-Basis Measurement

**Explanation:** An arbitrary qubit state is visualized on the Bloch sphere, showing its 3D orientation. Measuring in the X-basis involves rotating the qubit to the X-axis before measurement. This block helps students connect mathematical qubit descriptions to visual and probabilistic intuition.

In [30]:
# Bloch sphere via Statevector (works in Jupyter)
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import math

# Prepare an arbitrary single-qubit state: cos(theta/2)|0> + e^{i phi} sin(theta/2)|1>
theta = math.pi / 4
phi = math.pi / 3
qc_b = QuantumCircuit(1)
qc_b.u(theta, phi, 0, 0)  # u(theta, phi, lambda, qubit)

# Compute statevector
sv = Statevector.from_instruction(qc_b)
print('Statevector data:', sv.data)

# Bloch sphere visualization
plot_bloch_multivector(sv)

# --- Measure in X-basis: apply H before Z-measure ---
qc_x = QuantumCircuit(1, 1)
qc_x.u(theta, phi, 0, 0)  # prepare same state
qc_x.h(0)                 # rotate to X-basis
qc_x.measure(0, 0)

# Run on AerSimulator
sim = AerSimulator()
compiled = transpile(qc_x, sim)
job = sim.run(compiled, shots=1024)
result = job.result()
counts = result.get_counts()
print('X-basis measurement counts (|+> or |->):', counts)

Statevector data: [0.92387953+0.j         0.19134172+0.33141357j]
X-basis measurement counts (|+> or |->): {'1': 336, '0': 688}


----

**End of Module 1.**

Exercises: expand interference experiments to 4 qubits, implement custom multi-qubit phase oracle, and prepare mixed states via partial trace (advanced).